In [33]:
import json
import re
import pandas as pd
import xlwings as xw


In [34]:
with open("all_commands.json", "r") as file:
    all_commands = json.load(file)


In [35]:
commands = all_commands.copy()

for c in commands:
    if "Available Commands" in c:
        c["Available Commands"] = True

    else:
        c["Available Commands"] = False


In [36]:
temp_usage = []

for c in commands:
    if c["Available Commands"]:  # Final branch
        if "Usage" in c:
            temp_usage.extend(c["Usage"])

print(len(temp_usage))


92


In [37]:
commands


[{'command': 'cyber ',
  'Description': ['Bostrom Bootloader Hub'],
  'Usage': ['  cyber [command]'],
  'Available Commands': True,
  'Flags': ['  -h, --help                help for cyber',
   '      --home string         directory for config and data (default "/Users/user/.cyber")',
   '      --log_format string   The logging format (json|plain) (default "plain")',
   '      --log_level string    The logging level (trace|debug|info|warn|error|fatal|panic) (default "info")',
   '      --trace               print out full stack trace on errors',
   'Use "cyber [command] --help" for more information about a command.']},
 {'command': 'cyber collect-gentxs',
  'Description': ['Collect genesis txs and output a genesis.json file'],
  'Usage': ['  cyber collect-gentxs [flags]'],
  'Flags': ['      --gentx-dir string   override default "gentx" directory from which collect and execute genesis transactions; default [--home]/config/gentx/',
   '  -h, --help               help for collect-gentxs']

In [38]:
# n = 0
# for i in all_commands:
#     if i['command'] == 'cyber query evidence':
#         print(n)

#     n += 1

# del all_commands[40]


In [39]:
count = 0
for i in commands:
    all_flags = []
    all_flags.extend(i["Flags"])
    del i["Flags"]
    if "Global Flags" in i:
        all_flags.extend(i["Global Flags"])
        del i["Global Flags"]

    if all_flags != []:
        i["flags_parsed"] = []
        for b in all_flags:
            if match1 := re.match(
                r"\s\s+(?:-(?P<short>\w)[,\s]+)?(?:--(?P<long>[.\w-]+))\s(?:(?P<format>(uint32|ints|strings|int 64Slice|int|string|uint|float|bytesHex)))?\s*(?P<description>.*)?",
                b,
            ):
                i["flags_parsed"].append(
                    {
                        "long": match1.group("long"),
                        "short": match1.group("short"),
                        "format": match1.group("format"),
                        "description": match1.group("description"),
                    }
                )


In [40]:
for c in commands:
    c["flags_parsed"] = [f for f in c["flags_parsed"] if f["long"] != "help"]
    c["Usage"] = [
        l.replace("[flags]", "").replace(c["command"], "").strip() for l in c["Usage"]
    ]


In [41]:
commands


[{'command': 'cyber ',
  'Description': ['Bostrom Bootloader Hub'],
  'Usage': ['[command]'],
  'Available Commands': True,
  'flags_parsed': [{'long': 'home',
    'short': None,
    'format': 'string',
    'description': 'directory for config and data (default "/Users/user/.cyber")'},
   {'long': 'log_format',
    'short': None,
    'format': 'string',
    'description': 'The logging format (json|plain) (default "plain")'},
   {'long': 'log_level',
    'short': None,
    'format': 'string',
    'description': 'The logging level (trace|debug|info|warn|error|fatal|panic) (default "info")'},
   {'long': 'trace',
    'short': None,
    'format': None,
    'description': 'print out full stack trace on errors'}]},
 {'command': 'cyber collect-gentxs',
  'Description': ['Collect genesis txs and output a genesis.json file'],
  'Usage': [''],
  'Available Commands': False,
  'flags_parsed': [{'long': 'gentx-dir',
    'short': None,
    'format': 'string',
    'description': 'override default "g

In [62]:
df = pd.DataFrame(commands)
pd.json_normalize(df['flags_parsed'])

,0,1,2,3,4,5,6,7,8,9,...,46,47,48,49,50,51,52,53,54,55
0,"{'long': 'home', 'short': None, 'format': 'str...","{'long': 'log_format', 'short': None, 'format'...","{'long': 'log_level', 'short': None, 'format':...","{'long': 'trace', 'short': None, 'format': Non...",None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
1,"{'long': 'gentx-dir', 'short': None, 'format':...","{'long': 'home', 'short': None, 'format': 'str...","{'long': 'log_format', 'short': None, 'format'...","{'long': 'log_level', 'short': None, 'format':...","{'long': 'trace', 'short': None, 'format': Non...",None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
2,"{'long': 'home', 'short': None, 'format': 'str...","{'long': 'log_format', 'short': None, 'format'...","{'long': 'log_level', 'short': None, 'format':...","{'long': 'trace', 'short': None, 'format': Non...",None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
3,"{'long': 'account-number', 'short': 'a', 'form...","{'long': 'amount', 'short': None, 'format': 's...","{'long': 'broadcast-mode', 'short': 'b', 'form...","{'long': 'chain-id', 'short': None, 'format': ...","{'long': 'commission-max-change-rate', 'short'...","{'long': 'commission-max-rate', 'short': None,...","{'long': 'commission-rate', 'short': None, 'fo...","{'long': 'details', 'short': None, 'format': '...","{'long': 'dry-run', 'short': None, 'format': N...","{'long': 'fee-account', 'short': None, 'format...",...,None,None,None,None,None,None,None,None,None,None
4,"{'long': 'chain-id', 'short': None, 'format': ...","{'long': 'overwrite', 'short': 'o', 'format': ...","{'long': 'recover', 'short': None, 'format': N...","{'long': 'home', 'short': None, 'format': 'str...","{'long': 'log_format', 'short': None, 'format'...","{'long': 'log_level', 'short': None, 'format':...","{'long': 'trace', 'short': None, 'format': Non...",None,None,None,...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
279,"{'long': 'chain-id', 'short': None, 'format': ...","{'long': 'home', 'short': None, 'format': 'str...","{'long': 'log_format', 'short': None, 'format'...","{'long': 'log_level', 'short': None, 'format':...","{'long': 'trace', 'short': None, 'format': Non...",None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
280,"{'long': 'height', 'short': None, 'format': 'i...","{'long': 'latest-height', 'short': None, 'form...","{'long': 'node', 'short': None, 'format': 'str...","{'long': 'output', 'short': 'o', 'format': 'st...","{'long': 'prove', 'short': None, 'format': Non...","{'long': 'chain-id', 'short': None, 'format': ...","{'long': 'home', 'short': None, 'format': 'str...","{'long': 'log_format', 'short': None, 'format'...","{'long': 'log_level', 'short': None, 'format':...","{'long': 'trace', 'short': None, 'format': Non...",...,None,None,None,None,None,None,None,None,None,None
281,"{'long': 'height', 'short': None, 'format': 'i...","{'long': 'node', 'short': None, 'format': 'str...","{'long': 'output', 'short': 'o', 'format': 'st...","{'long': 'chain-id', 'short': None, 'format': ...","{'long': 'home', 'short': None, 'format': 'str...","{'long': 'log_format', 'short': None, 'format'...","{'long': 'log_level', 'short': None, 'format':...","{'long': 'trace', 'short': None, 'format': Non...",None,None,...,None,None,None,None,None,None,None,None,None,None
282,"{'long': 'height', 'short': None, 'format': 'i...","{'long': 'node', 'short': None, 'format': 'str...","{'long': 'output', 'short': 'o', 'format': 'st...","{'long': 'chain-id', 'short': None, 'format': ...","{'long': 'home', 'short': None, 'format': 'str...","{'long': 'log_format', 'short': None, 'format'...","{'long': 'log_level', 'short': None, 'format':...","{'long': 'trace', 'short': None, 'format': Non...",None,None,...,None,None,None,None,None,None,None,None,None,None


In [78]:
df = pd.DataFrame(commands).explode('flags_parsed').reset_index()
df['command'] = df['command'].apply(lambda x: str(x))
df2 = pd.json_normalize(df['flags_parsed'])

df3 = pd.concat([df[['command', 'Usage']], df2], axis=1)
xw.view(df3)


In [20]:
for_excel = pd.DataFrame(data=commands).applymap(lambda x: str(x))
xw.view(for_excel)


In [6]:
for i in all_commands:
    print(i["command"], "\n", i["flags_parsed"])


cyber  
 [{'long': 'help', 'short': 'h', 'format': None, 'description': 'help for cyber'}, {'long': 'home', 'short': None, 'format': 'string', 'description': 'directory for config and data (default "/Users/user/.cyber")'}, {'long': 'log_format', 'short': None, 'format': 'string', 'description': 'The logging format (json|plain) (default "plain")'}, {'long': 'log_level', 'short': None, 'format': 'string', 'description': 'The logging level (trace|debug|info|warn|error|fatal|panic) (default "info")'}, {'long': 'trace', 'short': None, 'format': None, 'description': 'print out full stack trace on errors'}]
cyber collect-gentxs 
 [{'long': 'gentx-dir', 'short': None, 'format': 'string', 'description': 'override default "gentx" directory from which collect and execute genesis transactions; default [--home]/config/gentx/'}, {'long': 'help', 'short': 'h', 'format': None, 'description': 'help for collect-gentxs'}, {'long': 'home', 'short': None, 'format': 'string', 'description': 'directory for c

In [12]:
from asyncore import write


types_nu_dict = {
    "uint": "int",
    "uint32": "int",
    # "float": "decimal",
    "float": "string",
    "strings": "string",
    "bytesHex": "string",
    "ints": "string",
}
lines = []

for c in all_commands:
    lines.append(f"\n# {c['Description']}\nextern '{c['command']}' [\n")
    # print(f"\n# {c['Description']}\nextern '{c['command']}' [")
    for f in c["flags_parsed"]:
        if "." not in f["long"]:
            long = f["long"]
            short = "(-" + f["short"] + ")" if f["short"] else ""
            f1 = types_nu_dict.get(f["format"], f["format"])
            format = ": " + f1 if f1 else ""
            description = f["description"]
            lines.append(f"\t--{long}{short}{format}\t\t# {description}\n")
            # print(f"\t--{long}{short}{format}\t\t# {description}")
    lines.append("]")
    # print("]")

with open("cyber-autocompletions.nu", "w") as file:
    file.writelines(lines)


In [8]:
import pandas as pd


In [ ]:
df = pd.DataFrame(all_commands).sort_values("command", ascending=False)


In [ ]:
path = "/Users/user/apps-files/github/cyber_completitions/json-files/cyber-query distribution.json"

with open(path, "r") as f:
    content = f.read()


In [ ]:
import pandas as pd

pd.DataFrame.from_records(json.loads(content))


In [ ]:
_list = []

for _file in os.scandir(
    "/Users/user/apps-files/github/cyber_completitions/json-files/"
):
    with open(_file, "r") as f:
        content = f.read()
        _list.append(json.loads(content))


In [ ]:
[i for i in _list if i != []]


In [ ]:
df = pd.concat([pd.DataFrame.from_records(i) for i in _list if i != []])
df


In [ ]:
df["com1"] = "cyber " + df["_command_root"] + " " + df["_command"]
df


In [ ]:
_l2 = df.loc[:, ["_description", "com1"]].to_dict("records")
_l2


In [ ]:
for l in _l2:
    print(f'# {l["_description"]}\nextern "{l["com1"]}" [] \n')


In [ ]:
for i in json.loads(content):
    print("\t", i["_command"], "\t\t#", i["_description"])
